# Data downloading

> Script to download LOBSTER sample data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp data_downloading

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import io
import os
import zipfile
from typing import Literal

import click
import requests

In [ ]:
# | exports
# | code-fold: true
CONTEXT_SETTINGS = dict(
    help_option_names=["-h", "--help"],
    token_normalize_func=lambda x: x.lower() if isinstance(x, str) else x,
)


@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-o", "--output-dir", required=False, help="location of output directory")
@click.option("-t", "--ticker", required=False, help="ticker")
@click.option("-l", "--levels", required=False, help="number of levels")
def get_sample_data(
    ticker: Literal["AMZN", "AAPL", "GOOG", "INTC", "MSFT"] = "AMZN",
    levels: Literal[1, 5, 10] = 5,
    output_dir=None,
):
    """Download and extract sample data from LOBSTER website."""
    SAMPLE_DATA_DATE = "2012-06-21"
    url = f"https://lobsterdata.com/info/sample/LOBSTER_SampleFile_{ticker}_{SAMPLE_DATA_DATE}_{levels}.zip"

    if output_dir is None:
        default_directory_name = "data/AMZN_2012-06-21_2-2012-06-21_5"
        output_dir = os.path.join(os.getcwd(), default_directory_name)

    os.makedirs(output_dir, exist_ok=True)

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to download data. HTTP Status Code: {response.status_code}")
        return

    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(output_dir)

To download and extract the [AMZN Lobster sample data](https://lobsterdata.com/info/DataSamples.php), run the following command in the directory of your choice.
```sh
$ get_sample_data
```
The output directory can also be specified with the optional -o flag
```sh
$ get_sample_data -o ./data/AMZN_2012-06-21_2-2012-06-21_5
```

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()